# what is in this version of the code
...


## Install Required Libraries

To ensure all necessary dependencies are installed


In [1]:
# # Upgrade LangChain and related modules
# ! pip install --upgrade langchain langchain-community langchain-chroma

# # Install LangChain integration for Groq
# ! pip install -qU langchain-groq

# # Install OpenAI integration for LangChain
# ! pip install langchain_openai

# # Install Hugging Face integration for LangChain
# ! pip install --upgrade langchain_huggingface

# # Install Unstructured and OpenPyXL for document processing
# ! pip install --upgrade unstructured openpyxl

# # Install Natural Language Toolkit (NLTK)
# ! pip install nltk

# # Install Sentence Transformers for embedding models
# ! pip install --upgrade --quiet langchain sentence_transformers

# # Install XLrd for reading Excel files
# ! pip install xlrd

# # Install xFormers for optimized transformer computations
# ! pip install xformers

# # Install Einops for tensor operations
# ! pip install einops

# # Install Hugging Face Transformers library
# ! pip install transformers

# # Upgrade Sentence Transformers
# ! pip install -U sentence-transformers

# # Install ChromaDB for vector database management
# ! pip install chromadb

## Import Required Libraries

These imports handle document processing, embeddings, caching, and vector storage.

In [2]:
# Chroma vector database and Groq chat model
from langchain_chroma import Chroma
from langchain_groq import ChatGroq

# Text splitting and document loaders
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredExcelLoader, DirectoryLoader

# Embeddings and language models
from langchain_community.embeddings import JinaEmbeddings
from sentence_transformers import SentenceTransformer

# Prompt templates and caching
from langchain.prompts import ChatPromptTemplate
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache

# Utilities
import os
import hashlib
from dotenv import load_dotenv
import nltk

/home/osamah/anaconda3/envs/masar_project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Download NLTK Resources

These datasets are required for tokenization, lemmatization, and part-of-speech tagging.

In [3]:
# Tokenization and lemmatization data
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Part-of-speech tagging data
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to /home/osamah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/osamah/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/osamah/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/osamah/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/osamah/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

## Data Indexing

### 1.Data Loading 
Load Documents from Directory

This code loads all `.txt` files from the specified directory and stores them in `docs`.

#### txt

In [4]:
loader = DirectoryLoader("../Data/", glob="*/*.txt")
docs = loader.load()

In [5]:
docs

[Document(metadata={'source': '../Data/laws/نظام الأحوال الشخصية.txt'}, page_content='نظام الأحوال الشخصية\n\n1443 هـ\n\nبسم الله الرحمن الرحيم\n\nمرسوم ملكي رقم (م/73) وتاريخ 1443/8/6هـ بعون الله تعالـى نحن سلمان بن عبدالعزيز آل سعود ملك المملكة العربية السعودية\n\nبناءً على المادة (السبعين) من النظام الأساسي للحكم، الصادر بالأمر الملكي رقم (أ / 90) بتاريخ 27 / 8 / 1412هـ. وبناءً على المادة (العشرين) من نظام مجلس الوزراء، الصادر بالأمر الملكي أرقم (أ / 13) بتاريخ 3 / 3 / 1414هـ. وبناءً على المادة (الثامنة عشرة) من نظام مجلس الشورى، الصادر بالأمر الملكي رقم (أ / 91) بتاريخ 27 / 8 / 1412هـ. وبعد الاطلاع على قراري مجلس الشورى رقم (145 / 27) بتاريخ 15 / 9 / 1442هـ، ورقم (100 / 18) بتاريخ 18 / 5 / 1443هـ. وبعد الاطلاع على قرار مجلس الوزراء رقم (429) بتاريخ 5 / 8 / 1443هـ.\n\nرسمنا بما هو آت:\n\nأولاً: الموافقة على نظام الأحوال الشخصية، بالصيغة المرافقة. ثانياً: يقصد بسن الرشد -لأغراض تطبيق نظام الأحوال الشخصية- تمام ثمانية عشر عاماً، وذلك إلى حين الموافقة على نظام المعاملات المدنية ونفاذه.

### 2.Data Splitting

In [6]:
# split the doc into smaller chunks i.e. chunk_size=512
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
chunks = text_splitter.split_documents(docs)

In [7]:
# Fixing the metadata if something is wrong with it
for chunk in chunks:
    for key, value in chunk.metadata.items():
        if isinstance(value, list):
            chunk.metadata[key] = ','.join(value)  # Convert list to a comma-separated string


In [8]:
chunks[5].page_content

'1443هـ، ورقم (1738) وتاريخ 3 / 8 / 1443هـ، المعدة في هيئة الخبراء بمجلس الوزراء. وبعد الاطلاع على التوصية المعدة في مجلس الشؤون الاقتصادية والتنمية رقم (12-35 / 42 / د) وتاريخ 3 / 9 / 1442هـ. وبعد النظر في قراري مجلس الشورى رقم (145 / 27) وتاريخ 15 / 9 / 1442ه، ورقم (100 / 18) وتاريخ 18 / 5 / 1443هـ. وبعد الاطلاع على توصية اللجنة العامة لمجلس الوزراء رقم (6823) وتاريخ 3 / 8 / 1443هـ.'

### 3.Data Embedding

In [9]:
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma

# Load Arabic sentence embedding model (no trust_remote_code needed)
embedding_model_name = "omarelshehy/Arabic-Retrieval-v1.0"
embedding_model = SentenceTransformer(embedding_model_name)

# Define a custom embedding wrapper for ChromaDB
class ArabicEmbeddings:
    def embed_documents(self, texts):
        return embedding_model.encode(texts).tolist()  # Convert NumPy array to list

    def embed_query(self, text):
        return embedding_model.encode([text])[0].tolist()  # Single text embedding

# Initialize embeddings
embeddings = ArabicEmbeddings()


### 4.Data Storing

In [10]:
# ! pip install chromadb

In [11]:
import os
import hashlib

# Define ChromaDB path
CHROMA_PATH = "vec_db"

# Function to generate a unique ID based on document content
def generate_id(text):
    return hashlib.md5(text.encode()).hexdigest()  # Hash-based unique ID

# Load existing database if it exists
if os.path.exists(CHROMA_PATH):
    db_chroma = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
    
    # Fetch existing document texts and compute their IDs
    stored_docs = db_chroma.get(include=["documents"])["documents"]  # Retrieve stored texts
    existing_ids = {generate_id(doc) for doc in stored_docs}  # Compute existing document IDs
else:
    db_chroma = None
    existing_ids = set()

# Prepare new documents with unique IDs
new_texts = []  # List to store new document texts
new_metadatas = []  # List to store corresponding metadata
new_ids = []  # List to store unique document IDs

for chunk in chunks:
    chunk_text = chunk.page_content  # Get text content
    doc_id = generate_id(chunk_text)  # Generate unique ID

    if doc_id not in existing_ids:  # Avoid re-adding duplicates
        new_texts.append(chunk_text)
        new_metadatas.append(chunk.metadata)
        new_ids.append(doc_id)

# Add only unique documents
if new_texts:
    if db_chroma is None:  
        # If DB was not initialized, create it with new documents
        db_chroma = Chroma.from_texts(new_texts, embeddings, metadatas=new_metadatas, ids=new_ids, persist_directory=CHROMA_PATH)
    else:
        # Correct method for adding new texts
        db_chroma.add_texts(new_texts, metadatas=new_metadatas, ids=new_ids)

# Persist database
if db_chroma:
    db_chroma.persist()

/tmp/ipykernel_1312062/880063558.py:13: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db_chroma = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
/tmp/ipykernel_1312062/880063558.py:47: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db_chroma.persist()


## Data Retrieval and Generation

### 1.Retrieval
Retrieve Context and Prepare Prompt

This code retrieves the top 20 most relevant chunks related to the user query using cosine similarity and formats the retrieved content into a structured prompt.


In [12]:
def retrieve_context_and_format_prompt(query: str, chat_id: str) -> str:
    """
    Retrieves the most relevant document chunks for the given query and formats them into a structured prompt.

    Args:
        query (str): The user's question.
        chat_id (str): The chat session ID.

    Returns:
        str: The formatted prompt ready for the LLM.
    """

    # Retrieve top 20 most relevant document chunks based on similarity search
    docs_chroma = db_chroma.similarity_search_with_score(query, k=20)

    # Extract text content from retrieved documents
    context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])

    # Define the structured prompt template
    PROMPT_TEMPLATE = """
    جاوب على السؤال بناءً على المعلومات القانونية والملفات المتاحة:

    **1- التحقق من المواد القانونية ذات الصلة:**
    - ابحث في الملفات المتاحة عن أي مواد قانونية مرتبطة بالقضية المطروحة.
    - اذكر رقم المادة ونصها كما هو مذكور في المصدر.
    - حدد الملفات التي تحتوي على هذه المواد القانونية.

    **2- فحص القضايا السابقة المشابهة:**
    - استخرج القضايا السابقة التي تشابه القضية الحالية من حيث الوقائع أو الأحكام.
    - قدم ملخصًا موجزًا عن كل قضية مشابهة، مع الإشارة إلى الفروقات أو التشابهات الجوهرية.
    - ذكر رقم الصفحة والملف الذي يحتوي على هذه القضايا.

    **3- استخراج النقاط المهمة التي قد تكون منسية في القضايا المشابهة:**
    - قم بتحليل الأنماط المتكررة في القضايا المشابهة وحدد أي نقاط مهمة غالبًا ما يتم تجاهلها.
    - قم بإبراز هذه النقاط وشرح مدى أهميتها في القضية الحالية.

    **4- تقديم إجابة واضحة ومنظمة دون أي إشارة إلى تعديلات لغوية:**
    - استخدم لغة دقيقة وسهلة الفهم دون الإشارة إلى أي تصحيحات أو تعديلات.
    - في حال وجود تعارض بين الأرقام المكتوبة بالكلمات والأرقام الرقمية، اعتمد على النص المكتوب بالكلمات.
    - أضف اسم الملف ورقم الصفحة أو المادة التي استندت إليها الإجابة لكل نقطة يتم ذكرها.
    - قدم ملخصًا نهائيًا بسيطًا يوضح الإجابة بشكل مباشر وواضح.

    جاوب على هذا السؤال: {question}

    اعطِ إجابة مفصلة ومنظمة وفق الخطوات المذكورة أعلاه.
    """

    return PROMPT_TEMPLATE, context_text

In [13]:
# print(context_text)
# print(query)

### 2.Generation
Generate Answer Using LLM (Function)

This function takes a formatted prompt and calls the `ChatGroq` model to generate a response.

### **Function Inputs:**
- `prompt (str)`: The fully formatted prompt containing context and query.

### **Function Output:**
- `str`: The generated response from the LLM.


In [14]:
# Load environment variables and retrieve API key
load_dotenv()
groq_api = os.getenv("GROQ_API_KEY")

# Ensure API key is available
if not groq_api:
    raise ValueError("GROQ_API_KEY not found in environment variables.")

# Initialize the LLM model
model = ChatGroq(model="llama-3.3-70b-versatile", api_key=groq_api)


In [21]:
# a function to use the prompt template and generate the answer based on the retrieved context
def generate_answer(prompt_template: str, context_text: str, query: str) -> str:
    """
    Generate an answer based on the prompt template and retrieved context.
    
    args:
        prompt_template (str): The prompt template to use for generating the answer.
        context_text (str): The retrieved context text.
        query (str): The user's query.

    returns:
        str: The generated answer.
    """
    prompt_template = ChatPromptTemplate.from_template(prompt_template)
    prompt = prompt_template.format(context=context_text, question=query)
    response_text = model.invoke(prompt)
    return response_text.content

In [23]:
query = "طلب ارجاع هبة"
chat_id = "01"

# retrieve context - top 50 most relevant (closests) chunks to the query vector
# (by default Langchain is using cosine distance metric)
docs_chroma = db_chroma.similarity_search_with_score(query, k=20)

# generate an answer based on given user query and retrieved context information
context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])

prompt_template, context_text = retrieve_context_and_format_prompt(query, chat_id)

# Call the function and get the response
response = generate_answer(prompt_template, context_text, query)
print(response)

جاوب على السؤال بناءً على المعلومات القانونية والملفات المتاحة:

**1- التحقق من المواد القانونية ذات الصلة:**

- ابحث في الملفات المتاحة عن أي مواد قانونية مرتبطة بالقضية المطروحة.
- فيما يتعلق بطلب ارجاع الهبة، يمكن الإشارة إلى المادة 544 من القانون المدني الذي ينص على: "تعتبر الهبة عقداً لازمياً بين الأطراف، ولا يمكن الرجوع فيه إلا في الحالات التي ينص عليها القانون" (ملف القانون المدني، الصفحة 120).
- كما تنص المادة 545 على: "يجب أن يكون الرجوع في الهبة متفقاً عليه بين الهبة والموهوب له، أو أن يكون هناك سبب قانوني يبرر هذا الرجوع" (ملف القانون المدني، الصفحة 121).

**2- فحص القضايا السابقة المشابهة:**

- استخرج القضايا السابقة التي تشابه القضية الحالية من حيث الوقائع أو الأحكام.
- في قضية "الموارد ضد عبد الله" (ملف القضايا السابقة، الصفحة 50-55)، تم النظر في طلب ارجاع هبة بسبب وجود غش في العقد. تم اعتبار الهبة لاغية بسبب عدم وجود إرادة حقيقية من قبل الموهوب له بسبب الغش.
- في قضية "الخليج ضد علي" (ملف القضايا السابقة، الصفحة 100-105)، تم رفض طلب ارجاع الهبة بسبب عدم وجود أي سبب قانون

In [17]:
# # load environment variables from .env file
# load_dotenv()

# # get GROQ API key from environment variable
# groq_api = os.getenv("GROQ_API_KEY")

In [18]:

# # this is an example of a user question (query)
# query = 'طلب استرجاع هبة'

# # retrieve context - top 50 most relevant (closests) chunks to the query vector
# # (by default Langchain is using cosine distance metric)
# docs_chroma = db_chroma.similarity_search_with_score(query, k=20)

# # generate an answer based on given user query and retrieved context information
# context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])


# PROMPT_TEMPLATE = """
# جاوب على السؤال بناءً على المحتوى التالي:
# {context}

# **1- التحقق من المواد القانونية ذات الصلة:**
# - ابحث في الملفات المتاحة عن أي مواد قانونية مرتبطة بالقضية المطروحة.
# - اذكر رقم المادة ونصها كما هو مذكور في المصدر.
# - حدد الملفات التي تحتوي على هذه المواد القانونية.

# **2- فحص القضايا السابقة المشابهة:**
# - استخرج القضايا السابقة التي تشابه القضية الحالية من حيث الوقائع أو الأحكام.
# - قدم ملخصًا موجزًا عن كل قضية مشابهة، مع الإشارة إلى الفروقات أو التشابهات الجوهرية.
# - ذكر رقم الصفحة والملف الذي يحتوي على هذه القضايا.

# **3- استخراج النقاط المهمة التي قد تكون منسية في القضايا المشابهة:**
# - قم بتحليل الأنماط المتكررة في القضايا المشابهة وحدد أي نقاط مهمة غالبًا ما يتم تجاهلها.
# - قم بإبراز هذه النقاط وشرح مدى أهميتها في القضية الحالية.

# **4- تقديم إجابة واضحة ومنظمة دون أي إشارة إلى تعديلات لغوية:**
# - استخدم لغة دقيقة وسهلة الفهم دون الإشارة إلى أي تصحيحات أو تعديلات.
# - في حال وجود تعارض بين الأرقام المكتوبة بالكلمات والأرقام الرقمية، اعتمد على النص المكتوب بالكلمات.
# - أضف اسم الملف ورقم الصفحة أو المادة التي استندت إليها الإجابة لكل نقطة يتم ذكرها.
# - قدم ملخصًا نهائيًا بسيطًا يوضح الإجابة بشكل مباشر وواضح.

# جاوب على هذا السؤال: {question}

# اعطِ إجابة مفصلة ومنظمة وفق الخطوات المذكورة أعلاه.
# """

In [19]:
# # # set_llm_cache(SQLiteCache(database_path=".langchain.db"))

# # load retrieved context and user query in the prompt template
# prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
# prompt = prompt_template.format(context=context_text, question=query)

# # call LLM model to generate the answer based on the given context and query
# model = ChatGroq(model="llama-3.3-70b-versatile", api_key=groq_api)
# response_text = model.invoke(prompt)

In [20]:
# print(response_text.content) # return response_text.content